<a href="https://colab.research.google.com/github/kchenTTP/python-series/blob/main/advanced_python_functions/Advanced_Python_Functions_Part_2_Closures_and_Decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Advanced Python Functions Part 2: Closures & Decorators**

In Part 1, we introduced higher-order functions and explored how functions can be passed as arguments. Now, let's take it a step further by looking at **higher-order functions that return functions as outputs**.

## **Table of Contents**

- [Closures](#scrollTo=bz-UTbfLev05)
- [Decorators](#scrollTo=Fkz86bj94ARo)
- [Common Decorators](#scrollTo=P3Aj9Hiisdwx)


In [1]:
# Import the libraries we will use in this class
from typing import Callable, Any, Union  # Type hint

## **Higher-Order Function Refresher**

Higher-order functions are functions that can either:

- Accept other functions as arguments
- Return functions as results
- Perform both of the above

Here's a function that takes a function as an argument:


In [2]:
# Encrypt strings based on provided encryption algorithm
def encrypt_message(message: str, encryption_algo: Callable[[str], str], **kwargs) -> str:
  return encryption_algo(message, **kwargs)

In [3]:
# decrypt strings based on provided decryption algorithm
def decrypt_message(message: str, decryption_algo: Callable[[str], str], **kwargs) -> str:
  return decryption_algo(message, **kwargs)

As you can see, both functions accept an encryption or decryption algorithm (a function) as an argument and execute it internally. Now, let's provide a simple encryption algorithm: the letter-number cipher.


In [4]:
def letter_number_cipher(message: str, decrypt: bool = False) -> str:
  """Encrypts or decrypts a message using a simple substitution cipher."""

  letter_to_number = {
      ' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7,
      'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14,
      'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21,
      'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26
  }
  number_to_letter = {v: k for k, v in letter_to_number.items()} # Swap key & value

  converted = []
  if decrypt:
    for num in message.split():
      converted.append(number_to_letter[int(num)])

    return ''.join(converted)

  else:
    # Encryption
    for char in message.lower():
      if char in letter_to_number:
        converted.append(str(letter_to_number[char]))

    return ' '.join(converted)

In [5]:
message = "Hello World!"
encrypted = encrypt_message(message, encryption_algo=letter_number_cipher)
print(encrypted)

message = "Houston, we have a problem."
encrypted = encrypt_message(message, encryption_algo=letter_number_cipher)
print(encrypted)

message = "20 15 0 2 5 0 15 18 0 14 15 20 0 20 15 0 2 5 0 20 8 1 20 0 9 19 0 20 8 5 0 17 21 5 19 20 9 15 14"
print(decrypt_message(message, decryption_algo=letter_number_cipher, decrypt = True))

8 5 12 12 15 0 23 15 18 12 4
8 15 21 19 20 15 14 0 23 5 0 8 1 22 5 0 1 0 16 18 15 2 12 5 13
to be or not to be that is the question


## **Closures**

Now, let's look at higher-order functions that return a function. Closures are functions that capture and carry their enclosing scope with them. Or, in simpler terms, they are:

> Functions that "remember" the environment (or the outer function) in which they were created, even if that environment is no longer in scope when the function is called.

Here's how it works:


In [6]:
def outside(x):
  def inside(y):
    return x + y
  return inside

# Using closures
add_five: Callable = outside(5)
result = add_five(10)
print(result)

# This is the equivalent the above code
print(outside(5)(10))

15
15


**Understanding Closures**

1. **Defining the Closure**:

  - We define an outer function `outside(x)`, which accepts a parameter `x`.
  - Inside `outside`, we define an inner function `inside(y)`, which takes a parameter `y` and returns the sum of `x` and `y`.
  - `inside` can access `x` because it is within the enclosing scope of `outside`.

2. **Creating a Closure**:

  - When we call `outside(5)`, Python creates a closure where `x` is set to `5`.
  - The call to `outside(5)` returns the `inside` function with `x` retained in its environment, allowing us to assign this closure to `add_five`.

3. **Using the Closure**:

  - We can now call `add_five(10)`, where `y` is `10`.
  - Since `add_five` retains `x = 5` in its environment, the result of `add_five(10)` is `5 + 10 = 15`.

4. **Equivalent Inline Call**:

  - We could also call `outside(5)(10)` directly, which immediately returns `15` without storing the closure in a variable.


### **Example 1: Weighted Average Calculator**

Let's look at an example of a higher order function that generates a function to compute the weighted average of a list of values.

**Formula**:

$$\text{Weighted Average} = \frac{\sum_{i=1}^{n} w_i \cdot x_i}{\sum_{i=1}^{n} w_i}$$

Here, we have 3 functions. The regular average, weighted average without using closure, and a weighted average with closure.


In [7]:
# Regular average for comparison
def average(values: list) -> float | int:
  return sum(values) / len(values)

# Weighted average without using closure
def weighted_average_no_closure(weights: list[float], values: list[float | int]) -> float | int:
  """Calculates the weighted average of given values using provided weights."""
  return sum(w * v for w, v in zip(weights, values)) / sum(weights)

# Weighted average closure example
def weighted_average(weights: list[float]) -> Callable[[list[float | int]], float | int]:
  """Returns a function that calculates the weighted average of given values."""

  def calculate_weighted_average(values: list[float | int]) -> float | int:
    """Calculates the weighted average for a given set of values."""
    return sum(w * v for w, v in zip(weights, values)) / sum(weights)

  return calculate_weighted_average

Let's first take a look at how using the weighted average without a closure would work:


In [8]:
# Weighted average without using closure
weights = [0.1, 0.3, 0.6]

values = [10, 20, 30]
print(weighted_average_no_closure(weights, values))

values = [40, 50, 60]
print(weighted_average_no_closure(weights, values))

25.0
55.0


As you can see, everytime you want to use it, you have to pass in **both** the `values` and the `weights`. Now, let's take a look at how it works *with* a closure this time.


In [9]:
weights = [0.1, 0.3, 0.6]

calculate_weighted_average = weighted_average(weights)

values = [10, 20, 30]
print(calculate_weighted_average(values))

values = [40, 50, 60]
print(calculate_weighted_average(values))

25.0
55.0


Instead of having to pass in the weights every time, our `calculate_weighted_average` function now *remembers* the weights and when we use it in the future, we only have to pass in the `values`. This is the power of closures.

You can also use it like this. Although, this is not how closures are typically used:


In [10]:
weights = [0.1, 0.3, 0.6]
values = [10, 20, 30]

print(weighted_average(weights)(values))

25.0


### **Example 2: API Endpoint Creator**

Let's look at another example.

Many web API services require you to include your API key (which you've obtained from them) with every request to authenticate yourself. Instead of manually adding the API key to each HTTP request, we can create a function that remembers the key and automatically includes it in the endpoint URL whenever we access a different endpoint.

<br>

> 📒 A typical api endpoint: https://www.youtube.com/watch?v=dQw4w9WgXcQ&t=1s


In [11]:
def api_endpoint_creator(api_key: str) -> Callable:
  def create_endpoint(endpoint: str, **params) -> str:
    url = f"{endpoint}?api_key={api_key}"
    for key, value in params.items():
      url += f"&{key}={value}"
    return url
  return create_endpoint

**Explanation**

1. **Function Definition**: The `api_endpoint_creator` function takes an `api_key` as a parameter and defines a nested function `create_endpoint`. This inner function constructs a URL based on the given `endpoint` and any additional parameters provided.

2. **Capturing State**: The `create_endpoint` function uses the `api_key` captured from the enclosing scope. Each time it is called, it builds a URL that includes this specific `api_key`.


In [12]:
# Create our first api endpoint creator with our first api key
api_key = "thisisanapikey123"
create_endpoint = api_endpoint_creator(api_key)

url_1 = create_endpoint("https://example.com/api-1", format="json", limit=10)
url_2 = create_endpoint("https://example.com/api-2", format="csv", limit=20)

print(url_1)
print(url_2)

https://example.com/api-1?api_key=thisisanapikey123&format=json&limit=10
https://example.com/api-2?api_key=thisisanapikey123&format=csv&limit=20


In [13]:
# Create our second api endpoint creator with our second api key
api_key = "admin-timmy-2000"
create_endpoint = api_endpoint_creator(api_key)

url_1 = create_endpoint("https://example.com/api-1", format="json", limit=10)
url_2 = create_endpoint("https://example.com/api-2", format="csv", limit=20)

print(url_1)
print(url_2)

https://example.com/api-1?api_key=admin-timmy-2000&format=json&limit=10
https://example.com/api-2?api_key=admin-timmy-2000&format=csv&limit=20


In both examples, we:

1. **Created an Endpoint**: We create a specific endpoint generator by calling `api_endpoint_creator` with the first API key. This returns a `create_endpoint` function tailored to use the provided `api_key`.

2. **Generated URLs**: When we call `create_endpoint` with different parameters, it constructs URLs that include the relevant API key and any other parameters provided.


### **Example 3: Email Template Generator**

Let's look at one last example of a closure to really drive the concept home.

Many email services let you create templates and personalize certain parts (like names or dates) for different recipients. This process, known as mail merging, can be replicated using closures. Let's see how we can use a closure to build a simple string templating function that customizes emails dynamically.


In [14]:
def template_creator(template: str) -> Callable:
  def create_email(**kwargs) -> str:
    return template.format(**kwargs)
  return create_email

**Explanation**

1. **Function Definition**: The `template_creator` function takes a `template` string as an argument, which contains any amount of placeholder text.

2. **Nested Closure**: Inside `template_creator`, we define the `create_email` function. This function takes any amount of parameters (for the placeholders) and uses the `template` string to generate a personalized email by formatting it with the provided values.

> 📒 For placeholder text, you need to use the exact same argument names you're going to pass to the enclosed function and surround it with curly brackets.


In [15]:
# Create an email template
template = "Hello {name},\n\n{message}\n\nBest regards,\nYour friends at TechConnect"
generate_email = template_creator(template)

# Using the email template generator
names = ["Thomas", "Mona", "Bob"]
for n in names:
  print(generate_email(name=n, message="I hope you are doing well!"))
  print("\n----\n")

Hello Thomas,

I hope you are doing well!

Best regards,
Your friends at TechConnect

----

Hello Mona,

I hope you are doing well!

Best regards,
Your friends at TechConnect

----

Hello Bob,

I hope you are doing well!

Best regards,
Your friends at TechConnect

----



In [16]:
# Create an email template
template = """Hi {name}!

Thank you for shopping at HappyPets.co

Here's your receipt:

{message}

Best,
Your Friends at HappyPets.co"""

generate_receipt = template_creator(template)

# Using the email template generator
names = ["Thomas", "Mona", "Bob"]
items = [
    {"Dog Treats": "$10", "Chew Toy": "$5"},
    {"Cat Food": "$15", "Scratching Post": "$30"},
    {"Bird Seed": "$8", "Bird Swing": "$12"}
]

for n, item_dict in zip(names, items):
    item_lines = [f"{item}: {price}" for item, price in item_dict.items()]
    message = "\n".join(item_lines)
    print(generate_receipt(name=n, message=message))
    print("\n----\n")

Hi Thomas!

Thank you for shopping at HappyPets.co

Here's your receipt:

Dog Treats: $10
Chew Toy: $5

Best,
Your Friends at HappyPets.co

----

Hi Mona!

Thank you for shopping at HappyPets.co

Here's your receipt:

Cat Food: $15
Scratching Post: $30

Best,
Your Friends at HappyPets.co

----

Hi Bob!

Thank you for shopping at HappyPets.co

Here's your receipt:

Bird Seed: $8
Bird Swing: $12

Best,
Your Friends at HappyPets.co

----



**Here, we've**:

1. **Created the Email Generator**: When we call `template_creator(template)`, it returns the `create_email` function, which is now configured to use the specific template we provided.

2. **Generated Emails**: Each time we call `generate_email`, we pass in the recipient's name and the message. The function uses the stored template to create the final email.


## **Decorators**

Now that we've seen functions that take another function as an argument and functions that return a function, let's look as something that does both.

**Decorators** allow you to modify or enhance the behavior of functions or methods without directly changing their source code. They are often used for logging, access control, caching, and other cross-cutting concerns.

<br>

**How Decorators Work**

A decorator is a function that takes another function as an argument and returns a new function that usually extends the behavior of the original function.

> You can either pass the function into a decorator when it's called, or you can apply a decorator to a function using the `@decorator_name` syntax before the function definition.


### **Example 1: Timing a Function**

First, let's look at a simple example of a decorator that measures the execution time of a function.

Let's say we want to measure the execution time of multiple functions to monitor performance. We could add timing code like this to each function individually:

In [17]:
import time

# No decorator example
def some_function():
  time.sleep(2)
  print("Function executed")


start = time.perf_counter()
some_function()
end = time.perf_counter()
print(f"Time taken: {end - start:.4f} seconds")

Function executed
Time taken: 2.0005 seconds


However, repeating this code can be tedious and error-prone. Instead, we can create a decorator that handles the timing for us. This way, we only write the timing functionality **once** and can easily apply it to any function with a simple decorator!


In [18]:
def timer(func: Callable) -> Callable:
  def wrapper(*args, **kwargs):
    start = time.perf_counter()
    rv = func(*args, **kwargs) # execute the passed in function here
    print(f"Function '{func.__name__}' executed in: {time.perf_counter() - start:.4f} seconds")
    return rv # return the return value of the passed in function
  return wrapper

**Explanation**

1. **Defining the Decorator**: The `timer` function takes a function `func` as an argument and defines a nested function `wrapper`. This `wrapper` function will enhance the original function's behavior.

2. **Measuring Execution Time**: Inside the `wrapper`, we record the start time, call the original function, and then record the end time. We calculate the duration, print it out, as well as return whatever the original function returns.

> 💡 Essentially, a decorator is a wrapper function around other functions

<br>

Now, we can use our `timer` decorator to measure the execution time of other functions. We have two ways to apply it:

1. **Directly Passing the Function**: We can pass a function into `timer` when calling it, like so:


In [19]:
from urllib.request import urlopen

def send_multiple_requests(lst: list[str]):
  for url in lst:
    urlopen(url)
    print(f"Opening: {url}")

timer(send_multiple_requests)(["https://www.google.com", "https://www.bing.com", "https://example.com/"])

Opening: https://www.google.com
Opening: https://www.bing.com
Opening: https://example.com/
Function 'send_multiple_requests' executed in: 0.3664 seconds


2. **Using `@timer` Syntax**: Alternatively, we can add `@timer` directly above the function definition. This automatically applies the `timer` decorator every time we call `send_multiple_requests`:


In [20]:
@timer
def send_multiple_requests(lst: list[str]):
  from urllib.request import urlopen
  for url in lst:
    urlopen(url)
    print(f"Opening: {url}")

send_multiple_requests([
    "https://www.google.com",
    "https://www.bing.com",
    "https://example.com/"
])

Opening: https://www.google.com
Opening: https://www.bing.com
Opening: https://example.com/
Function 'send_multiple_requests' executed in: 0.2652 seconds


Using the `@timer` syntax makes it simple to add or remove the decorator as needed, keeping the function clean and readable while still benefiting from the timing functionality.


### **Example 2: Ensuring a Function Only Executes Once**

Sometimes, you want a function (like a setup or initialization step) to run only once, even if it's called multiple times.

You could add a flag inside each function to track whether it's already run, but a cleaner way is to use a decorator like this:


In [21]:
def once(func):
  called = False
  result = None

  def wrapper(*args, **kwargs):
    nonlocal called, result
    if not called:
      result = func(*args, **kwargs)
      called = True
    return result
  return wrapper

**Explanation**

1. **Defining the once Decorator**: The `once` function takes the original function `func` and defines a `wrapper` function inside it. The `wrapper` function then adds the behavior of ensuring the original function is executed only once.

2. **Tracking Execution**: Inside the wrapper, we use a flag (`called`) to check if the function has already been executed. If it hasn't, we call the original function, set called to `True`, and store the result. On subsequent calls, the function simply returns the stored result without executing again.

<br>

Now, we can use our once decorator to ensure a function runs only once like so:


In [22]:
@once
def initialize():
  print("Setting up resources...")
  return {"status": "ready"}

# First call
initialize()

Setting up resources...


{'status': 'ready'}

In [23]:
# Second call just returns the result
initialize()

{'status': 'ready'}

This is really helpful for setup routines, connecting to a service, or loading configuration files, or any time you want to ensure a block of code runs just once, no matter how many times it's called.


### **Example 3: Function Argument Type Validator** (Additional Material)

❗ This is a more advanced example that involves multiple levels of nested functions. Feel free to skip it for now if you're not yet comfortable with higher-order functions, closures, and decorators.


In this example, we'll create a decorator that validates argument types. This decorator will accept a flexible number of argument type specifications and check if the provided arguments match the expected types. This allows us to enforce type safety without needing to add multiple checks inside the function.


In [24]:
from functools import wraps

def validate_args(**arg_types: type):
  def decorator(func: Callable) -> Callable:
    @wraps(func)
    def wrapper(*args, **kwargs):
      # Check positional arguments
      for arg, arg_type in zip(args, arg_types.values()):
        if not isinstance(arg, arg_type):
          raise TypeError(f"Expected argument '{arg}' of type {arg_type}, got {type(arg)}")

      # Check keyword argumemts
      for k, v in kwargs.items():
        if k in arg_types and not isinstance(v, arg_types[k]):
          raise TypeError(f"Expected argument '{k}' of type {arg_types[k]}, got {type(v)}")
      return func(*args, **kwargs)
    return wrapper
  return decorator

**Explanation**

1. **Decorator Definition**: The `validate_args` decorator takes keyword arguments, where each argument's name corresponds to the function's parameter name, and the value is the expected data type for that parameter.

2. **Creating the Wrapper**: Inside the `decorator` function, we define the `wrapper` function that validates each positional and keyword argument against the provided `arg_types`.

3. **Validating Arguments**:

  - For positional arguments, we use `zip` to match each argument with its corresponding type in `arg_types.values()`.
  - For keyword arguments, we check if each argument's name is in `arg_types` and validate its type.


> 📒 **NOTE**:
>
> `functools.wraps` is a decorator that copies the original function's metadata (like its name, docstring, and annotations) to the wrapper function in a decorator. This makes the wrapped function look and behave more like the original when inspected, logged, or documented.


In [25]:
@validate_args(a=int, b=int)
def add_two_numbers(a: int, b: int) -> int:
  return a + b

# Correct usage
add_two_numbers(1, 2)

# Incorrect argument datatypes: our decorator will catch them and give us a TypeError
add_two_numbers("a", "b")

TypeError: Expected argument 'a' of type <class 'int'>, got <class 'str'>

In [26]:
@validate_args(name=str, greeting=str)
def greet(name: str, greeting: str = "Hello") -> str:
  return f"{greeting}, {name}!"

# Correct usage
greet("Timmy")
greet("Timmy", "G'day")

# Incorrect argument datatypes
greet(1, "G'day")

TypeError: Expected argument '1' of type <class 'str'>, got <class 'int'>

In [27]:
@validate_args(name=str, age=int)
def introduce(name: str, age: int):
  print(f"My name is {name} and I am {age} years old.")

# Correct usage
introduce(name="Alice", age=30)  # Output: My name is Alice and I am 30 years old.

# Incorrect argument datatypes
introduce(name="Alice", age="thirty")

My name is Alice and I am 30 years old.


TypeError: Expected argument 'age' of type <class 'int'>, got <class 'str'>

> 💡 **Tip:**
>
> If a decorator doesn't take arguments, you only need one nested function (the `wrapper`). But if the decorator itself takes arguments, you'll need two nested functions, one to handle the decorator arguments and another as the actual wrapper.


## **Common Decorators (Additional Material)**

❗ This section requires some basic object-oriented programming (OOP) knowledge. Feel free to skip ahead and come back when you learn about attributes and properties in object-oriented programming.


### **`@functools.lru_cache`**

The `@functools.lru_cache` decorator caches (stores) function results based on arguments, so repeated calls with the same inputs return instantly instead of recomputing.


In [28]:
from functools import lru_cache
import random

# Simulate expensive database/API call
def fetch_user_data(user_id):
  time.sleep(2)  # Simulate network delay
  return {
      'id': user_id,
      'name': f'User_{user_id}',
      'score': random.randint(1, 100)
  }

# Without caching - slow every time
def get_user_score(user_id):
  data = fetch_user_data(user_id)
  return data['score']

# With caching - only slow the first time
@lru_cache(maxsize=128)
def get_user_score_cached(user_id):
  data = fetch_user_data(user_id)
  return data['score']

In [29]:
# Compare the difference
print("First calls:")

start = time.perf_counter()
score1 = get_user_score(123)
print(f"Without cache: {score1} (took {time.perf_counter() - start:.2f}s)")

start = time.perf_counter()
score2 = get_user_score_cached(123)
print(f"With cache: {score2} (took {time.perf_counter() - start:.2f}s)")

print("\nSecond calls:")

# Second calls - cached version will be instant
start = time.perf_counter()
score1 = get_user_score(123)
print(f"Without cache: {score1} (took {time.perf_counter() - start:.2f}s)")

start = time.perf_counter()
score2 = get_user_score_cached(123)
print(f"With cache: {score2} (took {time.perf_counter() - start:.2f}s)")

# Cache info
print("\nCache info:", get_user_score_cached.cache_info())

First calls:
Without cache: 1 (took 2.00s)
With cache: 94 (took 2.00s)

Second calls:
Without cache: 87 (took 2.00s)
With cache: 94 (took 0.00s)

Cache info: CacheInfo(hits=1, misses=1, maxsize=128, currsize=1)


📒 **Note:** You can also use more than one decorator at a time like this:

In [30]:
@timer
@lru_cache(maxsize=128)
def do_stuff(x: int, y: int) -> int:
  time.sleep(2)
  return x + y

_ = do_stuff(1, 2)
_ = do_stuff(1, 2)

Function 'do_stuff' executed in: 2.0002 seconds
Function 'do_stuff' executed in: 0.0000 seconds


You can see that cached functions consistently return the same result when given the same arguments, as the result is stored from previous calls. This caching mechanism speeds up execution by avoiding repeated calculations for identical inputs.


### **`@property`**

The `@property` decorator allows you to define a method that can be accessed like an attribute, making it a **getter** without explicitly calling it as a method.


In [31]:
class Person:
  def __init__(self, name: str, age: int) -> None:
    self._name = name
    self._age = age

  @property
  def name(self) -> str:
    return self._name

  @name.setter
  def name(self, name: str) -> None:
    self._name = name

  @property
  def age(self) -> int:
    return self._age

  @age.setter
  def age(self, age: int) -> None:
    self._age = age


# Example
person = Person("Timmy", 25)

# Use property getter to get properties
print("Old Properties:")
print(person.name)
print(person.age)

# Use property setter to set properties
person.name = "Jason"
person.age = 32

# Properties changed
print("---")
print("New Properties:")
print(person.name)
print(person.age)

Old Properties:
Timmy
25
---
New Properties:
Jason
32


### **`@dataclass`**

The `@dataclass` decorator automatically generates useful special methods for a class, including:

- `__init__` → Automatically creates an initializer based on class attributes.
- `__repr__` → Provides a readable string representation of the object.
- `__eq__` → Enables comparison between instances based on attribute values.
- ... and others like `__hash__`, `__post_init__`, and field defaults.

This makes defining simple data-holding classes much cleaner and more efficient.


In [32]:
from dataclasses import dataclass

@dataclass
class Point:
  x: float
  y: float

# Create a Point object
p1 = Point(1.0, 2.0)
p2 = Point(3.2, 4.0)
p3 = Point(1.0, 2.0)
print(p1)
print(p1 == p3)
print(p1 == p2)

Point(x=1.0, y=2.0)
True
False


The class created above is the exact same as the one created in the cell bellow, with a lot less boilerplate code:


In [33]:
class Point:
  x: float
  y: float

  def __init__(self, x: float, y: float):
    self.x = x
    self.y = y

  def __repr__(self) -> str:
    return f"Point(x={self.x}, y={self.y})"

  def __eq__(self, other: object) -> bool:
    if not isinstance(other, Point):
      return False
    return self.x == other.x and self.y == other.y


# Create a Point object
p1 = Point(1.0, 2.0)
p2 = Point(3.2, 4.0)
p3 = Point(1.0, 2.0)
print(p1)
print(p1 == p3)
print(p1 == p2)

Point(x=1.0, y=2.0)
True
False


## **Conclusion**

That's it for our class on closures and decorators. We explored how closures allow functions to retain state and how decorators provide a powerful way to modify functions dynamically.

For more information, check out these resources:
- [First-Class Functions and Closures in Python - freeCodeCamp](https://www.freecodecamp.org/news/first-class-functions-and-closures-in-python/)
- [Decorators in Python - DataCamp](https://www.datacamp.com/tutorial/decorators-python)
- [Primer on Python Decorators - Real Python](https://realpython.com/primer-on-python-decorators/)

<br>

In the next class, we'll look at `recursion` and how functions can call themselves to solve complex problems! 🐍
